In [1]:
from rdflib import Graph
from pydantic import BaseModel, Field
from typing import Optional
from api.src.schemas.authorities.authority import Uri, Variant, AdminMetadata, Element

In [34]:
class Subject(BaseModel):
    type: str 
    adminMetadata: AdminMetadata 
    elementList: list[Element]
    hasBroaderAuthority: Optional[list[Uri]]
    hasCloseExternalAuthority: Optional[list[Uri]]
    hasExactExternalAuthority: Optional[list[Uri]]
    hasNarrowerAuthority: Optional[list[Uri]]
    hasNarrowerExternalAuthority: Optional[list[Uri]]
    hasVariant: Optional[list[Variant]]
    subjectOf: Optional[list[Uri]]
    isMemberOfMADSCollection: str = Field(default='http://bibliokeia.com/authorities/Topic/')

In [54]:
authority = 'http://id.loc.gov/authorities/subjects/sh90003714-781'
graph = Graph()
graph.parse(f'{authority}.rdf')
graph.serialize('exemples/Geographic.ttl')

<Graph identifier=N4416f8daf6b64973b329bf42753e3923 (<class 'rdflib.graph.Graph'>)>

In [44]:
prefix = """PREFIX identifiers: <http://id.loc.gov/vocabulary/identifiers/>
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
  PREFIX skos: <http://www.w3.org/2004/02/skos/core#>"""

qNote = f"""{prefix}
  SELECT ?note WHERE {{ 
      <{uri}> madsrdf:note ?note .
       }}"""

r = graph.query(qNote)
r.bindings


[{rdflib.term.Variable('note'): rdflib.term.Literal('Use as a topical subdivision under occupational groups and types of employees.')}]

In [4]:
def getUri(obj, metadado, authority, graph):
    query = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            SELECT * WHERE  {{
              <{authority}> madsrdf:{metadado} ?value .
              ?value madsrdf:authoritativeLabel ?label
              }}"""
    response = graph.query(query)
    if len(response.bindings) > 0:
        uris = list()
        for i in response.bindings:
          uri = {
                  "value": i.get('value').n3().replace('<','').replace('>',''),
              "label": {
                "value": i.get('label').value,
                "lang": i.get('label').language
              } }
          uris.append(uri)
        obj[metadado] = uris
    return obj

In [51]:

from api.src.function.authorities.getUri import GetUri


def ParserSubject(graph, authority):
      
#   authority = 'http://id.loc.gov/authorities/subjects/sh2018002121'
  prefix = """PREFIX identifiers: <http://id.loc.gov/vocabulary/identifiers/>
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>"""

  # Type
  qtype = f"""{prefix}
  SELECT ?type WHERE {{ 
      <{authority}> rdf:type ?type .
      FILTER ( ?type != madsrdf:Authority ) 
      FILTER ( ?type != skos:Concept )
       }}"""
  r = graph.query(qtype)
  tipo = r.bindings[0].get('type').split("#")[1]

  # identifiers
  # qlccn = f"""{prefix}
  # SELECT ?o WHERE {{ 
  #     <{authority}> identifiers:lccn ?o }}"""
  # [lccn] = [i[0].value for i in graph.query(qlccn) ]

  # adminMetadata
  adminMetadata = {
      "assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
      "identifiedBy": [ {
          "assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
          "value": lccn        
      }]
  }
  # ElementList
  qElementList = f"""{prefix}
  SELECT ?elementValue ?type WHERE  {{
    <{authority}> madsrdf:elementList ?o .
    ?o rdf:rest* ?node .
    ?node rdf:first ?e .
    ?e madsrdf:elementValue ?elementValue .
    ?e rdf:type ?type
    }}"""
  r = graph.query(qElementList)
  elementList = list()
  for i in r.bindings:
      element = {
            "type": i.get('type').split("#")[1],
          "elementValue": {
            "value":  i.get('elementValue').value,
            "lang": i.get('elementValue').language
          }
        }
      elementList.append(element)

  obj = {
     "type": tipo,
      "adminMetadata": adminMetadata,
      "elementList": elementList
  }

# hasVariant
  qVariant = f"""{prefix}
  SELECT ?typeVariant ?typeElement ?elementValue WHERE  {{
	<{authority}> madsrdf:hasVariant ?variant .
  ?variant rdf:type ?typeVariant .
  ?variant madsrdf:elementList ?elementList .
  ?elementList rdf:rest* ?node .
    ?node rdf:first ?e .
    ?e madsrdf:elementValue ?elementValue .
	?e rdf:type ?typeElement .
  FILTER ( ?typeVariant != madsrdf:Variant )
  }}"""
  r = graph.query(qVariant)
  if len(r.bindings) > 0:
    variants = list()
    for i in r.bindings:
      variant = {
          'type': i.get('typeVariant').split("#")[1],
          'elementList': [{
              'type': i.get('typeElement').split("#")[1],
              'elementValue': {
                  'value': i.get('elementValue').value,
                  'lang': i.get('elementValue').language
              }
          }]
      }
      variants.append(variant)
    obj['hasVariant'] = variants

  # URIS
  metadados = ['hasBroaderAuthority', 
               'hasCloseExternalAuthority',
    'hasExactExternalAuthority',
    'hasNarrowerAuthority',
    'hasNarrowerExternalAuthority']
  for metadado in metadados:
     obj = GetUri(obj, metadado, authority, graph)
  
  response = Subject(**obj)
  return response


In [55]:
response = ParserSubject(graph, authority)
response.dict()

ValueError: not enough values to unpack (expected 1, got 0)

{'type': 'Topic',
 'adminMetadata': {'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
  'descriptionModifier': 'http://id.loc.gov/vocabulary/organizations/brmninpa',
  'creationDate': datetime.date(2023, 5, 4),
  'descriptionLanguage': 'http://id.loc.gov/vocabulary/languages/por',
  'generationProcess': 'BiblioKeia v.1',
  'generationDate': '2023-05-04T08:32:48',
  'identifiedBy': [{'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
    'value': 'sh 85113459'}],
  'status': {'value': 'mstatus:new', 'label': 'novo'}},
 'elementList': [{'type': 'TopicElement',
   'elementValue': {'value': 'Revolutionary literature, Portuguese',
    'lang': 'en'}}],
 'hasBroaderAuthority': [{'value': 'http://id.loc.gov/authorities/subjects/sh85105349',
   'label': {'value': 'Portuguese literature', 'lang': 'en'}}],
 'hasCloseExternalAuthority': [{'value': 'http://id.worldcat.org/fast/1096634',
   'label': {'value': 'Revolutionary literature, Portuguese', 'lang': None}}],
 'hasExact